<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Методы машинного обучения</center></h1>
<h2><center>Выявление сообществ на сети, структурная схожесть</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

In [ ]:
import networkx as nx

## Выявление сообществ с помощью иерархической кластеризации
* Загрузим граф из `nested_partition.gml`. Это граф, построенный с помощью [генератора](https://sites.google.com/site/santofortunato/inthepress2) Benchmark сетей для тестов алгоритмов выявления сообществ.
* Визуализируем матрицу смежности графа с помощью метода `plt.spy(A)` (и саму сеть, если получится)
* Рассчитаем реализованные в `nexworkx` меры схожести вершин

По большому счету, полученные схожести можно подавать на вход любому алгоритму кластеризации, который изначально может работать со мерами близости (например аггломеративная кластеризация)

In [ ]:
g = nx.read_gml('./data/nested_partition.gml', label='id')

In [ ]:
g.number_of_nodes()

In [ ]:
A = nx.adj_matrix(g)
plt.spy(A, markersize=2)

In [ ]:
# Что бы мы увидили в реальной жизни
idx = np.random.permutation(range(128))
i = np.ix_(idx, idx)
plt.spy(A[i], markersize=2)

In [ ]:
# Все положительные похожести по Жаккару
jac = nx.jaccard_coefficient(g)
jac = np.array(list(jac))

In [ ]:
jac

In [ ]:
# Немного магии, чтобы получить из этого матрицу попарных СХОЖЕСТЕЙ по жаккару
from scipy.sparse import coo_matrix

# Переводим все в формат разреженной матрицы
D = coo_matrix((jac[:,2], (jac[:,0], jac[:,1])), shape=(128,128))

# Делаем ее симметричной
D = D+D.T

# Делаем ее плотной и вычитаем из 1, чтобы получить РАССТОЯНИЕ
D = 1 - D.toarray() - np.eye(128)

In [ ]:
plt.imshow(D)

In [ ]:
# Попробуем посчитать на этой матрице рассстойний иерархическую кластеризацию
from scipy.cluster.hierarchy import dendrogram, fcluster, linkage
from scipy.spatial.distance import squareform, pdist

In [ ]:
d = squareform(D) # преобразование матрицы в вектор расстояний
Z = linkage(d, method='average')
dend = dendrogram(Z)
labels = fcluster(Z, 4, criterion='maxclust')

In [ ]:
true_labeling = list(nx.get_node_attributes(g, 'label').values())

In [ ]:
layout = nx.layout.spectral_layout(g)
nx.draw(g, pos=layout, node_color=true_labeling,)

In [ ]:
nx.draw(g, pos=layout, node_color=labels)

## Label propagation

Методы для выявления сообществ, которые реализованы в networkX можно найти [тут](https://networkx.github.io/documentation/stable/reference/algorithms/community.html#module-networkx.algorithms.community.community_utils)

In [ ]:
lp_partition = nx.community.asyn_lpa_communities(g) # должен быть seed

In [ ]:
lp_partition = list(lp_partition) # разбиение на сообщества 

In [ ]:
len(lp_partition)

In [ ]:
# Функция, которая из разбиения получает разметку для каждого объекта
def get_labeling_from_partition(partition):
    
    all_nodes = set()
    for cluster in partition:
        all_nodes |= cluster
    num_nodes = len(all_nodes)
    labeling = np.ones((num_nodes,), dtype=int)

    for label, ids in enumerate(partition):
        ids = list(ids)
        labeling[ids] = label
        
    return labeling

In [ ]:
lp_labeling = get_labeling_from_partition(lp_partition)

In [ ]:
nx.draw(g, pos=layout, node_color=lp_labeling)

## Edge betweenness

In [ ]:
eb_partitions = nx.community.girvan_newman(g)

In [ ]:
k = 4 # Вернем разбиение на k сообщества
for partition in eb_partitions:
    if len(partition) == k:
        break

In [ ]:
eb_partition = list(partition) # разбиение на сообщества 

In [ ]:
eb_labeling = get_labeling_from_partition(eb_partition)

In [ ]:
eb_labeling

In [ ]:
nx.draw(g, pos=layout, node_color=eb_labeling)

### Modularity

In [ ]:
# Просто функция, которая считает модулярность
from itertools import product
def modularity(G, communities, weight='weight'):
    multigraph = G.is_multigraph()
    directed = G.is_directed()
    m = G.size(weight=weight)
    if directed:
        out_degree = dict(G.out_degree(weight=weight))
        in_degree = dict(G.in_degree(weight=weight))
        norm = 1 / m
    else:
        out_degree = dict(G.degree(weight=weight))
        in_degree = out_degree
        norm = 1 / (2 * m)

    def val(u, v):
        try:
            if multigraph:
                w = sum(d.get(weight, 1) for k, d in G[u][v].items())
            else:
                w = G[u][v].get(weight, 1)
        except KeyError:
            w = 0
        # Double count self-loops if the graph is undirected.
        if u == v and not directed:
            w *= 2
        return w - in_degree[u] * out_degree[v] * norm

    Q = sum(val(u, v) for c in communities for u, v in product(c, repeat=2))
    return Q * norm

In [ ]:
eb_partitions = nx.community.girvan_newman(g)

In [ ]:
# Посчитаем модулярность для разбиений на 1,2,..10 сообществ
for partition in eb_partitions:
    num_com = len(partition)
    if num_com < 10:
        mod = modularity(g, partition)
        print('For {} communities modularity = {}'.format(num_com, mod))
    else:
        break

# Пример расчета Асортативности

В файле `Princeton.gml` содержится граф дружбы студентов соответствующих, собранных в 2005 году из Facebook. Каждая вершина обладает следующими аттрибутами:
* Факультет
* Пол
* Основное направление подготовки
* Дополнительный направление подготовки (если есть)
* Общежитие проживания, домашнее проживание
* Год поступления
* Школа

Пропуски помечены значением `0`.

#### Задание
Посчитайте ассортативность для каждого из аттрибутов и ассортативность по степени узлов.

Сравните результаты между сетями и проинтерпретируйте их.

In [ ]:
!head ./data/Princeton.gml

In [ ]:
g = nx.read_gml('./data/Princeton.gml', label='id')

In [ ]:
attributes = g.node[0]

In [ ]:
attributes = g.node[0].keys()

In [ ]:
for attr in attributes:
    print('{} assortativity = {}'.format(attr, 
                                         nx.assortativity.attribute_assortativity_coefficient(g, attr)))

In [ ]:
# Матрица перемешивания e_ij
E = nx.assortativity.attribute_mixing_matrix(g, 'year')

In [ ]:
plt.imshow(E)

In [ ]:
nx.assortativity.degree_assortativity_coefficient(g)